#  Untersuchen eines fertigen künstlichen Neuronales Netzes (KNN)

**Funktionsweise eines fertigen Künstlichen Neuronalen Netzwerks**

Wir wollen verstehen wie Künstliche Neuronale Netzwerke (KNN) funktionieren und wie man sie konfiguriert und trainiert, um verschiedene Aufgaben zu lösen. Folgende Fragen sind dafür relevant:

    1. Welche Art von Aufgaben kann ein KNN lösen?
    
    2. Wie funktioniert ein fertiges KNN?
    
    3. Wie wird ein KNN mit Hilfe von Daten trainiert? 
    
    4. Wie konfiguriert man ein KNN passend zu einer Problemstellung und einem Datensatz?
    
In diesem Notebook wollen wir die **ersten beiden Fragen** beantworten und uns anschauen wie eine fertiges Neuronales Netzwerk aufgebaut ist und wie es funktioniert. Dafür schauen wir uns ein sehr einfaches Anwendungsbeispiel und ein dazu passendes fertiges künstliches neuronales Netzwerk an. 

Später, in weiteren Notebooks schauen wir uns dann das Lernprinzip und die Konfiguration eines KNN an, um auch die **dritte und vierte Frage** zu beantworten.
    

# Klassifikationsproblem

Mit KNN können Klassifikationsaufgaben gelöst werden, wie es für verschiendene Methoden des Machine Learning typisch ist. Bei einer Klassifikationsaufgabe geht es darum ein Objekt anhand einer Menge verschiedener Merkmale zu erkennen und einer Klasse zuzuordnen. 





Ein sehr einfaches Beispiel für ein Klassifikationsproblem ist die Unterscheidung von Insekten. Dabei haben wir nur die zwei Klassen <font color=blue>**Marienkäfer**</font> und <font color=red>**Raupe**</font>. Die Merkmale, die man zur Erkennung heranziehen könnte sind vielfältig. Man könnte z. B. Farben, Größe, Fortbewegungsgeschwindigket und vieles mehr anschauen. Da wir aber ein möglichst einfaches Beispiel betrachten wollen schauen wir uns nur "Breite" und "Länge" der Insekten an. 

<td> 
    <img src="Bilder\marienkaefer.jpg" alt="Drawing" style="width: 300px; float: left;"  hspace=40 /> 
    <img src="Bilder\raupe.jpg" alt="Drawing" style="width: 300px; float: left;" hspace=40/>
</td>


Wenn man sich überlegt wie die Breite und Länge von Marienkäfern und Raupen aussieht, könnten diese Merkmale genügen um eine passende Zuordnung zu treffen. Beispielhaft können wir uns einmal die Maße von zwei Raupen und einem Marienkäfer anschauen: 

<td> 
    <img src="Bilder\InsektenMessen.jpg" alt="Drawing" style="width: 800px; float: left;" /> 
    
</td>


# Datenbeispiel

In [2]:
#Bibliothek pandas zur Datenverarbeitung importieren
import pandas as pd

#Bibliotheken plotly und vufflinks zur Datenvisualisierung importieren
from plotly.offline import iplot
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline()

#Ein quadratsiches Layout für plotly-Graphiken wird definiert 
square_layout = go.Layout(xaxis={'title':'Breite','range':[-0.01,1.01]},
                   yaxis={'title':'Länge','range':[-0.01,1.01]},
                   height=800,
                   width=800)


/Users/kbovi/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning:

A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1



In [3]:
#Datensatz einlesen
df_käfer = pd.read_csv('Kaefer.csv', sep=';')

df_käfer#.head()

,Breite,Länge,Insekt
0,0.48,0.38,Marienkäfer
1,0.32,0.74,Raupe
2,0.49,0.27,Marienkäfer
3,0.58,0.46,Marienkäfer
4,0.11,0.58,Raupe
...,...,...,...
195,0.57,0.37,Marienkäfer
196,0.10,0.54,Raupe
197,0.39,0.81,Raupe
198,0.56,0.29,Marienkäfer



In der Folgenden Zelle wird dieser Datensatz als Streudiagramm (Scatterchart) visualisiert. 

Mit diesem Streudiagramm können wir zunächst analysieren, wie die Eigenschaft der unterschiedlichen Insekten sind.

In [4]:
df_käfer.iplot('scatter', mode = 'markers',  layout = square_layout,
               x = 'Breite', 
               y = 'Länge',
               categories = 'Insekt')

/Users/kbovi/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/cufflinks/plotlytools.py:807: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.

/Users/kbovi/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/cufflinks/plotlytools.py:810: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.




<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Im Datensatz <code>df_käfer</code> sind 100 Marienkäfer und 100 Raupen erfasst worden. Beschreibe was sich anhand des Streudiagramms über Raupen und Marienkäfer aussagen lässt. 
    </div> 
</div>







ANWORT HIER EINFÜGEN


___

Die Aufgabe diese beiden Insektentypen anhand von Länge und Breite zu klassifizieren ist nicht besonders kompliziert, aber sehr gut geeignet, um daran nachzuvollziehen wie ein relativ einfaches fertiges Künstliches Neuronales Netzwerk arbeitet.

# Das künstliche neuronale Netzwerk
Im folgenden wollen wir nachvollziehen, wie ein Künstliches Neuronalen Netz (KNN) Daten über Länge und Breite nach dem sogenannten **Feed-Forward-Prinzip** verarbeitet und jeweils einer Insektenklasse zuordnen kann.

Das von uns betrachtete KNN kann Marienkäfer und Raupen anhand von Länge und Breite klassifizieren. 

<a id="Abschnitt4_1"> </a>
## Aufbau

<div class="panel panel-success">
    <div class="panel-heading">
        <b> Eklärung - Aufbau des künstlichen Neuronalen Netzwerks</b>
    </div>
    <div class="panel-body">
Wir betrachten im folgenden ein Künstliches Neuronales Netz, das aus <b>4 Neuronen</b> besteht. Diese 4 Neuronen sind angeordnet in <b>3 Layern</b>.

Der <font color=blue>**Input Layer**</font> besteht aus 2 Neuronen <code>I1</code> und <code>I2</code>, der <font color=green>**Hidden Layer**</font> besteht aus nur einem Neuron <code>H1</code> und der <font color=red>**Output Layer**</font> besteht ebenfalls aus nur einem Neuron <code>O1</code>.

Das Neuronale Netz bekommt als Eingabe (Input) 2 Werte, nämlich die Länge und Breite eines Insekts. Diese Werte werden dann durch das Neuronale Netz verarbeitet und die Neuronen in den unterschiedlichen Layern werden dadurch entweder <font color=blue> **aktiviert** </font> oder <font color=red> **nicht aktiviert** </font>. Die Entscheidung, die das neuronale Netz trifft hängt von der Aktivierung des letzten (Output-) Neurons ab.
        
* Wenn das letzte Neuron aktiviert ist, wählt das neuronale Netz den Ausgabewert <font color=blue>**Marienkäfer**</font>
      
* Wenn das letzte Neuron nicht aktiviert ist, wählt das neuronale Netz die Ausgabewert <font color=red>**Raupe**</font>
        
Dieser Ausgabewert gibt an welches Insekt das künstliche neuronale Netzwerk hinter den Eingabewerten (Breite & Länge) vermutet.
    </div> 
</div>

In [5]:
#Anzeigen der Daten von Insekt Nummer 199
df_käfer.loc[199]

Breite     0.38
Länge      0.93
Insekt    Raupe
Name: 199, dtype: object

Z.B. kann man die Werte für Breite und Länge von Insekt Nummer 199 eingeben und das Neuronale Netz erkennt, dass es sich um eine Raupe handelt. 

Für eine erste Visualisierung dieses Vorgangs starte das folgende Video. Die Verarbeitungsschritte und die Aktivierung der Neuronen des Netzwerks werden darin zunächst farblich dargestellt. Was genau dabei passiert schauen wir uns danach an.

<video src="Videos\FeedForward_Vid1.mp4" width="600" 
    autobuffer autoplay controls>
</video>

<div class="panel panel-danger">
    <div class="panel-heading">
        <b>Wichtige Erkenntnisse</b>
    </div>
    <div class="panel-body">
Wir können festhalten:

* Das künstliche neuronale Netzwerk kann anhand von Breite und Länge entscheiden welches Insekt vorliegt. 
        
* Die Entscheidung für die Insektenklassen <font color=blue>Marienkäfer</font> oder <font color=red>Raupe</font> hängt davon ab ob das letzte (Output-) Neuron <font color=blue> **aktiviert** </font> oder <font color=red> **nicht aktiviert** </font> ist.

Wir wissen noch nicht:

* Was genau machen die Neuronen? 
</div> 


<a id="Abschnitt4_2"> </a>
## Was ist ein künstliches Neuron?



<div class="panel panel-success">
    <div class="panel-heading">
        <b> Eklärung - Was ist ein künstliches Neuron?</b>
    </div>
    <div class="panel-body">
Die künstliche Neuronen die wir betrachten sind inspiriert von realen Neuronen im menschlichen Gehirn. Allerdings ist das was die künstlichen Neuronen machen nicht besonders kompliziert. Vereinfacht kann man sich merken:
        
<br><br>
<center style="font-size:20px"><i> Ein künstliches Neuron ist ein  <b>Ding, das eine Zahl enthält</b>.</i> 
</center>  
<br><br>
        
Genauer gesagt können Zahlen zwischen 1 und -1 in einem Neuron enthalten sein. An dieser Zahl kann man die Aktivierung des Neurons ablesen. Ein Neuron gilt als 

*  <font color=blue> **aktiviert** </font>, wenn der Wert zwischen 0 und 1 liegt 

und als 

* <font color=red> **nicht aktiviert** </font>, wenn der Wert zwischen -1 und 0 liegt.

Je näher der Wert also an der 1 ist desto aktiver ist das Neuron und umgekehrt je näher der Wert an der -1 ist desto inaktiver ist das Neuron.

Die tatsächliche Aktivierung eines Neurons hängt von den eingehenden Signale ab und kann daher unterschiedliche Zahlen annehmen, je nachdem welchen Input man in das neuronale Netzwerk eingibt.
        
</div> 



        

Im folgenden Video schauen wir uns einmal an, wie die Aktivierungen der einzelnen Neuronen bei dem vorherigen Beispiel aussehen. 
Die Neuronen färben sich blau wenn sie aktiviert sind und rot wenn nicht aktiviert sind.

<video src="Videos\FeedForward_Vid2.mp4" width="600" 
    autobuffer autoplay controls>
</video>

<div class="panel panel-danger">
    <div class="panel-heading">
        <b>Wichtige Erkenntnisse</b>
    </div>
    <div class="panel-body">

Wir können festhalten:

* Künstliche Neuronen enthalten Zahlenwerte zwischen -1 und 1.       



* Die ersten beiden Neuronen im Input Layer übernehmen einfach die Werte der Eingaben (Länge und Breite) als Aktivierungswerte.



* Das letzte Neuron im Outputlayer bestimmt welche Klasse ausgewählt wird. Wenn das letzte Neuron <font color=red>nicht aktiviert</font> ist wird die Klasse <font color=red>Raupe</font>  gewählt. Wenn das letzte Neuron <font color=blue> aktiviert </font> ist, wird die Klasse <font color=blue>Marienkäfer</font> gewählt.


Wir wissen noch nicht:


* Wie kommen die Aktivierungswerte von den Neuronen im Hidden Layer und im Output Layer zustande?

        
       
</div> 







<div class="panel panel-warning">
    <div class="panel-heading">
        <b>Prüfe dich selbst: </b>
    </div>
    <div class="panel-body">

        
* Wann ist ein Neuron aktiviert und wann ist es nicht aktiviert? ( Nachlesen: <a href="#Abschnitt4_2">4.2. Neuron</a>)

* Was sind Input Layer, Hidden Layer und Output Layer? (Nachlesen: <a href="#Abschnitt4_1">4.1. Aufbau eines KNN</a>)

</div> 





## Aktivierung eines Neurons

<div class="panel panel-success">
    <div class="panel-heading">
        <b> Eklärung - Aktivierung eines Neurons</b>
    </div>
    <div class="panel-body">

Die Aktivierung eines Neurons hängt immer davon ab welche Aktivierungswerte die Neuronen aus dem vorherigen Layer übermitteln. Die so eingehenden Signale werden addiert und je nachdem wie hoch die Summe der eingehenden Signale ist wird das Neuron aktiviert oder nicht aktiviert.        
        
Allerdings werden nicht einfach die Aktivierungswerte der vorherigen Neuronen addiert, sondern diese werden beim übermitteln über Verbidnungsstrecken (Kanten) zwischen den Neuronen noch verändert. An jeder Kante steht ein sogenanntes **Kantengewicht**. 

<br><br>        
<center style="font-size:20px"><i> Kantengewichte sind <b>Vorfaktoren, die mit dem Wert des vorherigen Neurons multipliziert werden</b>. </i> 
</center>   
<br><br>
        
Die eingehenden Signale mit den Kantengewichten multipliziert und anschließend aufaddiert werden. Wenn diese Summe über einem bestimmten Schwellwert liegt wird das nächste Neuron aktiviert. Wenn die Summe unter dem Schwellwert liegt wird das nächste Neuron nicht aktiviert. Der konkrete Wert der Aktivierung wird durch eine **Aktivierungsfunktion** bestimmt.
        
<br><br>        
<center style="font-size:20px"><i>Die Aktivierungsfunktion erhält als Eingabewert die Summe der eingehenden Signale und <b>berechnet daraus daraus den Wert der Aktivierung des Neurons</b>. </i> 
</center>   
<br><br>
        
        
        
</div> 



        

Im folgenden Video wird die Aktivierung des Neurons im Hidden Layer einmal beispielhaft dargestellt. Folgende Schritte gehören dazu:

* Die Kantengewichte, die gezeigt werden haben die Werte 3.6 und -3.7


* Die eingehenden Signale werden mit den Kantengewichten multipliziert


* Es wird die Summe der eingehenden Signale gebildet


* Die Aktivierungsfunktion wird auf die Summe der Signale angewandt

<video src="Videos\FeedForward_Vid3_1.mp4" width="600" 
    autobuffer autoplay controls>
</video>

### Aktivierungsfunktion

Nun schauen wir uns noch einmal die Aktivierungsfunktion etwas genauer an. Wir wollen nun herausfinden wie die Aktivierungsfunktion Eingabewerte umwandelt. Wir haben gesehen, dass z.B. für den Eingabewert -2.07 ein Aktivierungswert von -0.96 herauskommt.

In [6]:
#import der Aktivierungsfunktions
from numpy import tanh as aktivierung

In [7]:
#Anwenden der Aktivierungsfunktion - Der Eingabewert entspricht der Summer der eingehenden Signale

aktivierung(-2.07)

-0.9686534238679029


<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Ändere in der vorherigen Codezelle den Eingabewert für die Aktivierungsfunktion. Probiere verschiedenen Eingabewerte aus und finde einen Wert für den...
        
        a) ... als Ausgabewert 1.0 herauskommt.
        b) ... als Ausgabewert -1.0 herauskommt.
        c) ... als Ausgabewert 0.0 herauskommt.
        d) ... als Ausgabewert ca. 0.5 herauskommt.
        e) ... als Ausgabewert 5.0 herauskommt.

Notiere deine Eingabewerte in der nächsten Zelle. Falls du keinen Eingabewert findest, begründe warum das so ist.    
    </div> 
</div>




Eingabewerte hier notieren:

a)

b)

c)

d)

e)


In der folgenden Zelle wird der Graph der Aktivierungsfunktion für Eingabewerte von -10 bis 10 visualisiert.

In [8]:
#Den Code in dieser Zelle musst du nicht verstehen - Wichtig ist die erzeugt Visualisierung
#Layout der Graphik festlegen
aktivierung_layout = go.Layout(xaxis={'title':'Input','range':[-10.0,10.0]},
                               yaxis={'title':'Aktivierung','range':[-1.5,1.5]},
                               height=500,
                               width=1000)

#Wertetabelle erzeugen
df_aktivierung = pd.DataFrame()
df_aktivierung['Input']=[x/100 for x in range(-1000, 1000)]
df_aktivierung['Aktivierung']=[aktivierung(x/100) for x in range(-1000, 1000)]

#Werte als Graphik Visualisieren
df_aktivierung.iplot(x='Input', y='Aktivierung', layout = aktivierung_layout)

<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Welchen Eingabewerte kann man als Schwellwert bezeichnen, oberhalb dessen das Neuron feuert (aktiviert ist)? 
        
Schwellwert bedeutet, dass unterhalb des Schwellwerts das Neuron nicht aktiviert ist und oberhalb des Schwellwerts das Neuron aktiviert ist. 
    </div> 
</div>



Antwort hier einfügen.

<div class="panel-group">
    <div class= "panel panel-default">
        <div class = "panel-heading">
            <h4 class="panel-title">
                <a data-toggle="collapse" href="#testhilfe1"> Weitere Information Aktivierungsfunktion </a>
            </h4>
        </div>
        <div id="testhilfe1" class="panel-collapse collapse">
            <div class="panel-body"> Es gibt verschiedene Aktivierungsfunktionen die für Neuronale Netze genutzt werden. Die Aktivierungsfunktion, die in unserem KNN zum Einsatz kommt ist der Tangens Hyperbolicus oder auch kurz <code>tanh</code>.

Die Funktionsgleich lautet:

$$\large{tanh(x) = 1 - \frac{2}{e^{2x}+1}}$$

Das sieht ziemlich kompliziert aus! Mit dem genauen Funktionsterm müssen wir uns aber nicht näher auseinandersetzen, da wir die Funktion <code>tanh</code> ganz einfach aus der <code>numpy</code>  Bibliothek importieren können.
            </div>
        </div>
    </div>
</div>


# Den Output berechnen - Das Feed-Forward-Prinzip

### Kantengewichte 

Außer den Neuronen und dem Aufbau der Layer sind noch weitere Dinge wichtig, damit des Künstliche Neuronale Netz funktionieren kann. Die **Kantengewichte** sind Zahlenwerte, die an jeder Verbindung zwischen zwei Neuronen stehen. Im oben abgebildeten KNN gibt es die drei Kantengewichte <code>w1</code>,
<code>w2</code> und <code>w3</code>. Diese konkreten Werte für unser KNN stehen in der folgenden Zelle.


In [ ]:
#Festlegung der Kantengewichte

w1 = 3.6

w2 = -3.7

w3 = 3.2

Die Kantengewichte wirken zunächst willkürlich. Diese Kantengewichte sind aber das Ergebnis eines Lernprozesses basierend auf dem Datensatz <code>df_käfer</code>. Wie genau der Lernprozess funktioniert schauen wir uns zunächst nicht an. Um später den Lernprozess verstehen zu können, schauen wir uns zunächst an wie ein fertiges Künstliches Neuronales Netz arbeitet.

Neben den **Neuronen** und den **Kantengewichten** ist noch eine weitere Sache wichtig für das Künstliche Neuronale Netz. Jedes Neuron besitzt noch eine **Aktivierungsfunktion**.

In [ ]:
#import der Aktivierungsfunktions
from numpy import tanh

Wir haben weiter oben die Kantengewichte  <code>w1</code>, <code>w2</code> und <code>w3</code> definiert und kennen nun unsere Aktivierungsfunktion <code>tanh</code>.

Als nächstes schauen wir uns den Verarbeitungsprozess des KNN an. Wir schauen uns also genauer an wie aus den Inputwerten <code>x1</code> und <code>x2</code> ein Outputwert <code>y</code> wird. Dies funktioniert nach dem sogenannten Feed-Forward-Prinzip, was soviel heißt wie "vorwärts durchlaufen".

<video src="Videos\knn_feed_forward.mp4" width="600" 
    autobuffer autoplay controls>

</video>

In [ ]:
df_käfer.head()

Wir geben zunächst als Inputwerte die Breite und die Länge des ersten Käfers ein. Der erste Käfer ist ein Marienkäfer.



## Input Layer

Der Input Layer ist immer etwas besonders, da er keine besondere Aktivierungsfunktion enthält. D. h. in den Neuronen des Input Layers werden die Inputdaten einfach weitergegeben und nicht durch eine Aktivierungsfunktion wie z. B. tanh verarbeitet.


In [ ]:
#Breite
x1 = 0.38

#Länge
x2 = 0.93

## Hidden Layer

Nun durchlaufen die Outputwerte der Neuronen <code>I1</code> und <code>I2</code> die Kanten mit den Gewichten <code>w1</code> und <code>w2</code>. 

An jeder Kante wird der Output des vorherigen Layers mit dem Kantengewicht multipliziert. Die Summe über alle Kanten bildet den Input für das Neuron des nächsten Layers. Die Summe wird als Eingabewert für die Aktivierungsfunktion (tanh) genutzt.



In [ ]:
#Summe der Signale mit Kantengewichten
w1 * x1 + w2 * x2


In [ ]:
#Aktivierung des Neurons im Hidden Layer
tanh(w1 * x1 + w2 * x2)
  

## Output Layer

Nun durchlaufen der Outputwert des Neurons <code>H1</code> die Kante mit dem Gewicht <code>w3</code>. 

Da nur eine Kante zwischen Hidden Layer und Output Layer existiert muss dieses mal keine Summe gebildet werden, sondern nur ein Produkt.


In [ ]:
#Summe der Signale mit Kantengewichten
w3 * tanh( w1 * x1 + w2 * x2) 

In [ ]:
#Aktivierung des Neurons im Output Layer
tanh(w3 * tanh( w1 * x1 + w2 * x2) )
  

Der korrekte Output ist <code>-0.9959516357629201</code>. Falls bei dir ein anderer Output herauskommt muss in den vorherigen Schritten etwas schief gelaufen sein. 

Falls das Ergebnis nicht übereinstimmt überprüfe deine vorherigen Rechnungen.

<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Ändere weiter oben die Eingabewerte des neuronalen Netzes (x1 und x2). Suche dir eine andere Insekt aus dem Datensatz und verifiziere, dass auch dafür ein sinnvolles Ergebnis herauskommt.
    </div> 
</div>



# Den Output des Neuronalen Netzes visualisieren


<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Damit man nicht jedes mal mehrere Zellen ausführen muss um einen Output zu berechnen benötigen wir eine Funktion.

Implementiere diese Funktion, die als Input <code>x1</code> und <code>x2</code> bekommt und <code>y</code> als Output liefert.

Definiere innerhalb der Funktion die Gewichte <code>w1</code>, <code>w2</code> und <code>w3</code>
    </div> 
</div>




In [ ]:
def knn_output(x1, x2):
    w1 = 3.7
    w2 = -3.6
    w3 = 3.2
    y = tanh(w3*tanh(w1*x1+w2*x2))
    return y

In [ ]:
#zum testen
knn_output(0.38, 0.93)

Falls die Funktion <code>knn_output</code> korrekt funktioniert, kann mit dem ausführen der folgenden Zelle eine Übersicht über verschiedene Outputs erstellt werden. 

In [ ]:
#Übersicht über verschiedene Outputs erstellen
ergebnisse = pd.DataFrame(index = [y/10 for y in reversed(range(10))])

for x in range(10):
    ergebnisse[str(x/10)] = [knn_output(y/10,x/10) for y in reversed(range(10))]

ergebnisse#.T

## Outputwerte als Heatmap Visualisieren

Um die Übersicht über die Outputwerte etwas anschaulicher zu machen, kann man alle Outputs auch als "Heatmap" anzeigen lassen.

Auf der x-Achse steht die Breite, auf der y-Achse die Länge und in der Mitte ist die Heatmap unterschiedlich eingefärbt, je nachdem welchen Output das KNN liefert.

In [ ]:
ergebnisse.iplot('heatmap', xTitle='Breite', yTitle ='Länge', colorscale='RdBu')

## Feinere Übersicht über die Outputs des KNN

In [ ]:
heatmap=pd.DataFrame()

for x in range(100):
    heatmap[str(x/100)] = [knn_output(y/100,x/100) for y in range(100)]

heatmap.index = [y/100 for y in range(100)]

heatmap.iplot('heatmap', xTitle='Breite', yTitle ='Länge', colorscale='RdBu', layout=square_layout)
#heatmap.iplot('heatmap', xTitle='Breite', yTitle ='Länge', colorscale='RdBu')

<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Versuche die Heatmap zu interpretieren. Ist Sie sinnvoll? 

Schaue dir dafür auch nochmal das Streudiagramm des Datensatzes <code>df_käfer</code> weiter oben an.

(Hinweis: Wenn du mit der Maus über die Heatmap fährst werden verschiedene Werte angezeigt)
    </div> 
</div>





ANTWORT HIER EINFÜGEN

## Performance des KNN bewerten

### Vorhersagegenauigkeit / Fehlklassifikationsrate

In [ ]:
#Erstelle Spalte mit interpretierten Outputs des KNN
Outputs=[]

for i in range(len(df_käfer)):
    
    output = knn_output(df_käfer.iloc[i]['Breite'],df_käfer.iloc[i]['Länge'])
    
    if output > 0:
        Outputs.append('Marienkäfer')
    elif output <= 0:
        Outputs.append('Raupe')

        
df_käfer_test = df_käfer.copy()
df_käfer_test['Outputs'] = Outputs

df_käfer_test.head()

In [ ]:
#Vergleiche Outputs mit tatsächlichen Werten
df_käfer_test[df_käfer_test['Insekt'] != df_käfer_test['Outputs']]

In [ ]:
#Berechne wie viele Beispiele korrekt klassifiziert werden 
sum(df_käfer_test['Insekt'] == df_käfer_test['Outputs'])

### Mittlere Numerische Abweichung

In [ ]:
Outputs=[]

for i in range(len(df_käfer)):
    
    Outputs.append(knn_output(df_käfer.iloc[i]['Breite'],df_käfer.iloc[i]['Länge']))
    
df_käfer_test2 = df_käfer.replace(['Marienkäfer', 'Raupe'],[-1,1])
df_käfer_test2['Outputs'] = Outputs

df_käfer_test2.head()

In [ ]:
df_käfer_test2['Insekt'] - df_käfer_test2['Outputs']

In [ ]:
sum(abs(df_käfer_test2['Insekt'] - df_käfer_test2['Outputs']))

In [ ]:
abs(df_käfer_test2['Insekt'] - df_käfer_test2['Outputs']).mean()